In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
import multiprocessing


from sklearn.datasets import load_boston, load_breast_cancer,load_iris
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1.나이브 베이스 분류기(Naive Bayes Classification)

* 베이즈 정리(조건부 확률)를 적용한 확률적 분류 알고리즘
* 모든 특성들이 독립임을 가정 (naive 가정)
* 입력 특성에 따라 3개의 분류기 존재
  * 가우시안 나이브 베이즈 분류기
  * 베르누이 나이브 베이즈 분류기
  * 다항 나이브 베이즈 분류기

# 2.나이브 베이즈 분류기의 확률 모델

* 나이브 베이즈는 조건부 확률 모델
* *N*개의 특성을 나타내는 벡터 **x**를 입력 받아 k개의 가능한 확률적 결과를 출력

\begin{equation}
p(C_k | x_1,...,x_n)
\end{equation}

* 위의 식에 베이즈 정리를 적용하면 다음과 같음

\begin{equation}
p(C_k | \textbf{x}) = \frac{p(C_k)p(\textbf{x}|C_k)}{p(\textbf{x})}
\end{equation}

* 위의 식에서 분자만이 출력 값에 영향을 받기 때문에 분모 부분을 상수로 취급할 수 있음

\begin{equation}
\begin{split}
p(C_k | \textbf{x}) & \propto p(C_k)p(\textbf{x}|C_k) \\
& \propto p(C_k, x_1, ..., x_n)
\end{split}
\end{equation}

* 위의 식을 연쇄 법칙을 사용해 다음과 같이 쓸 수 있음

\begin{equation}
\begin{split}
p(C_k, x_1, ..., x_n) & = p(C_k)p(x_1, ..., x_n | C_k) \\
& = p(C_k)p(x_1 | C_k)p(x_2, ..., x_n | C_k, x_1) \\
& = p(C_k)p(x_1 | C_k)p(x_2 | C_k, x_1)p(x_3, ..., x_n | C_k, x_1, x_2) \\
& = p(C_k)p(x_1 | C_k)p(x_2 | C_k, x_1)...p(x_n | C_k, x_1, x_2, ..., x_{n-1})
\end{split}
\end{equation}

* 나이브 베이즈 분류기는 모든 특성이 독립이라고 가정하기 때문에 위의 식을 다음과 같이 쓸 수 있음

\begin{equation}
\begin{split}
p(C_k, x_1, ..., x_n) & \propto p(C_k)p(x_1|C_k)p(x_2|C_k)...p(x_n|C_k) \\
& \propto p(C_k) \pro


- 정수형 또는 실수형: GaussianNB (1, 3, 2), (0.1, 0.9, 1.5)
- 정수형: MultinomialNB (1, 3, 2)
- 바이너리형: BernoulliNB (0,1,0)
- alpha : smoothing


In [ ]:
weather=['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast','Sunny','Sunny', 'Rainy','Sunny','Overcast','Overcast','Rainy']
temp=['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']
play=['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']

df = pd.DataFrame( [weather,temp])
df = df.T
df.columns = ['날씨', '온도']
df['외출여부'] = play
df

,날씨,온도,외출여부
0,Sunny,Hot,No
1,Sunny,Hot,No
2,Overcast,Hot,Yes
3,Rainy,Mild,Yes
4,Rainy,Cool,Yes
5,Rainy,Cool,No
6,Overcast,Cool,Yes
7,Sunny,Mild,No
8,Sunny,Cool,Yes
9,Rainy,Mild,Yes


In [ ]:
x_data = df.iloc[:,:-1]
y_data = df.iloc[:, -1]

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: 외출여부, dtype: object

In [ ]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
# make_column_transformer
ct = make_column_transformer( (OneHotEncoder(), ['날씨','온도']), remainder = 'passthrough' )

In [ ]:
# pipline 만들기
model_pip = make_pipeline( ct, BernoulliNB() )
model_pip.fit(x_data, y_data)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['날씨', '온도'])])),
                ('bernoullinb', BernoulliNB())])

In [ ]:
testdf = pd.DataFrame([['Sunny', 'Hot']], columns=['날씨', '온도'])

In [ ]:
model_pip.predict( pd.DataFrame([['Sunny', 'Hot']], columns=['날씨', '온도']) )

array(['No'], dtype='<U3')

In [ ]:
# 토마토( 6, 4   ) ==> 분류
df = pd.read_excel('datas/음식.xlsx')
df

,이름,당도,아삭함,종류
0,사과,10,9,과일
1,베이컨,1,4,단백질
2,바나나,10,1,과일
3,당근,5,10,채소
4,샐러리,3,10,채소
5,치즈,1,1,단백질
6,오이,2,8,채소
7,생선,3,1,단백질
8,포도,8,5,과일
9,콩,3,7,채소


In [ ]:
x_data = df.iloc[:,1:-1]
y_data = df.iloc[:,-1]

In [ ]:
x_data

,당도,아삭함
0,10,9
1,1,4
2,10,1
3,5,10
4,3,10
5,1,1
6,2,8
7,3,1
8,8,5
9,3,7


In [ ]:
y_data

0      과일
1     단백질
2      과일
3      채소
4      채소
5     단백질
6      채소
7     단백질
8      과일
9      채소
10     채소
11    단백질
12     과일
13     과일
14    단백질
Name: 종류, dtype: object

In [ ]:
ct = make_column_transformer( (OneHotEncoder(), ['이름','종류']), remainder = 'passthrough' )

In [ ]:
# 문류가 3개의 종류 이기 때문에 MultinomialNB() 이나 GaussianNB()를 써야함
model = make_pipeline(GaussianNB())
model.fit(x_data, y_data)
model.predict(pd.DataFrame([[6,4]],columns=['당도','아삭함']))

array(['과일'], dtype='<U3')